In [1]:
import math

from nltk import sent_tokenize, word_tokenize, PorterStemmer
from nltk.corpus import stopwords 

In [5]:
text_content ='''The Russian tycoon Oleg Tinkov has denounced Moscow’s “massacre” in Ukraine and urged the west to help end “this insane war”.

Offering some of the strongest criticism by a prominent Russian of the Kremlin’s military action, Tinkov claimed online that 90% of Russians were “against this war” and called Russia’s forces a “shit army”.

Tinkov is one of Russia’s best-known entrepreneurs and founded Tinkoff Bank in 2006. He has been based outside Russia in recent years.

“Waking up with a hangover, the generals realised that they have a shit army,” Tinkov, 54, wrote on Instagram.

“And how will the army be good, if everything else in the country is shit and mired in nepotism, sycophancy and servility?”

The businessman, who has been targeted by western sanctions, added: “I don’t see a SINGLE beneficiary of this insane war! Innocent people and soldiers are dying.”

Switching to English, Tinkov, who stepped down as chairman of Tinkoff Bank in 2020, wrote: “Dear ‘collective west’ please give Mr Putin a clear exit to save his face and stop this massacre. Please be more rational and humanitarian.”

He posted his statement on the 55th day of Moscow’s military campaign, with Russian forces unleashing a major new offensive in the eastern Donbas region.

“90 percent of Russians ARE against this war!” Tinkov said. “Of course there are morons who draw Z but 10 percent of any country are morons,” he said, referring to what has become a Russian symbol of support for the war.

He added that Kremlin officials were “in shock” that they and their children would no longer be able to spend their summer holidays on the Mediterranean. “Businessmen are trying to rescue what’s left of their property.”

Russian authorities have sought to strangle dissent over Moscow’s invasion of Ukraine, imposing prison terms of up to 15 years for publishing what the Kremlin calls “fake news” about the army.

Tinkoff Bank said in a statement that it would not comment on Tinkov’s “private opinion”, saying he no longer took decisions regarding operations across companies under the Tinkoff brand. “He is not a Tinkoff employee.”

Russia’s president, Vladimir Putin, sent troops to Ukraine on 24 February. The conflict has killed thousands of people and displaced more than 12 million in the largest refugee crisis in Europe since the second world war.'''

In [6]:
text= text_content


In [7]:
sentences = sent_tokenize(text)
total_documents = len(sentences)

In [8]:
def _score_sentences(tf_idf_matrix) -> dict:
    """
    score a sentence by its word's TF
    Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.
    :rtype: dict
    """

    sentenceValue = {}

    for sent, f_table in tf_idf_matrix.items():
        total_score_per_sentence = 0

        count_words_in_sentence = len(f_table)
        for word, score in f_table.items():
            total_score_per_sentence += score

        sentenceValue[sent] = total_score_per_sentence / count_words_in_sentence

    return sentenceValue

In [19]:
def _generate_summary(sentences, sentenceValue, threshold):
    sentence_count = 0
    summary = ''

    for sentence in sentences:
        if sentence[:15] in sentenceValue and sentenceValue[sentence[:15]] >= (threshold):
            summary += " " + sentence
            sentence_count += 1

    return summary

In [17]:
def _find_average_score(sentenceValue) -> int:
    """
    Find the average score from the sentence value dictionary
    :rtype: int
    """
    sumValues = 0
    for entry in sentenceValue:
        sumValues += sentenceValue[entry]

    # Average value of a sentence from original summary_text
    average = (sumValues / len(sentenceValue))

    return average

In [9]:
def _create_tf_idf_matrix(tf_matrix, idf_matrix):
    tf_idf_matrix = {}

    for (sent1, f_table1), (sent2, f_table2) in zip(tf_matrix.items(), idf_matrix.items()):

        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(f_table1.items(),
                                                    f_table2.items()):  # here, keys are the same in both the table
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sent1] = tf_idf_table

    return tf_idf_matrix

In [10]:
def _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents):
    idf_matrix = {}

    for sent, f_table in freq_matrix.items():
        idf_table = {}

        for word in f_table.keys():
            idf_table[word] = math.log10(total_documents / float(count_doc_per_words[word]))

        idf_matrix[sent] = idf_table

    return idf_matrix

In [13]:
def _create_documents_per_words(freq_matrix):
    word_per_doc_table = {}

    for sent, f_table in freq_matrix.items():
        for word, count in f_table.items():
            if word in word_per_doc_table:
                word_per_doc_table[word] += 1
            else:
                word_per_doc_table[word] = 1

    return word_per_doc_table

In [14]:
def _create_tf_matrix(freq_matrix):
    tf_matrix = {}

    for sent, f_table in freq_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(f_table)
        for word, count in f_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sent] = tf_table

    return tf_matrix

In [15]:
def _create_frequency_matrix(sentences):
    frequency_matrix = {}
    stopWords = set(stopwords.words("english"))
    ps = PorterStemmer()

    for sent in sentences:
        freq_table = {}
        words = word_tokenize(sent)
        for word in words:
            word = word.lower()
            word = ps.stem(word)
            if word in stopWords:
                continue

            if word in freq_table:
                freq_table[word] += 1
            else:
                freq_table[word] = 1

        frequency_matrix[sent[:15]] = freq_table

    return frequency_matrix

In [11]:
print(text)

The Russian tycoon Oleg Tinkov has denounced Moscow’s “massacre” in Ukraine and urged the west to help end “this insane war”.

Offering some of the strongest criticism by a prominent Russian of the Kremlin’s military action, Tinkov claimed online that 90% of Russians were “against this war” and called Russia’s forces a “shit army”.

Tinkov is one of Russia’s best-known entrepreneurs and founded Tinkoff Bank in 2006. He has been based outside Russia in recent years.

“Waking up with a hangover, the generals realised that they have a shit army,” Tinkov, 54, wrote on Instagram.

“And how will the army be good, if everything else in the country is shit and mired in nepotism, sycophancy and servility?”

The businessman, who has been targeted by western sanctions, added: “I don’t see a SINGLE beneficiary of this insane war! Innocent people and soldiers are dying.”

Switching to English, Tinkov, who stepped down as chairman of Tinkoff Bank in 2020, wrote: “Dear ‘collective west’ please give M

In [20]:
sentences = sent_tokenize(text)
total_documents = len(sentences)


freq_matrix = _create_frequency_matrix(sentences)



tf_matrix = _create_tf_matrix(freq_matrix)

count_doc_per_words = _create_documents_per_words(freq_matrix)


idf_matrix = _create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)

tf_idf_matrix = _create_tf_idf_matrix(tf_matrix, idf_matrix)

sentence_scores = _score_sentences(tf_idf_matrix)

threshold = _find_average_score(sentence_scores)


summary = _generate_summary(sentences, sentence_scores, 0.7 * threshold)
print(summary)


 The Russian tycoon Oleg Tinkov has denounced Moscow’s “massacre” in Ukraine and urged the west to help end “this insane war”. Tinkov is one of Russia’s best-known entrepreneurs and founded Tinkoff Bank in 2006. He has been based outside Russia in recent years. “Waking up with a hangover, the generals realised that they have a shit army,” Tinkov, 54, wrote on Instagram. Please be more rational and humanitarian.”

He posted his statement on the 55th day of Moscow’s military campaign, with Russian forces unleashing a major new offensive in the eastern Donbas region. “90 percent of Russians ARE against this war!” Tinkov said. “Of course there are morons who draw Z but 10 percent of any country are morons,” he said, referring to what has become a Russian symbol of support for the war. He added that Kremlin officials were “in shock” that they and their children would no longer be able to spend their summer holidays on the Mediterranean. Tinkoff Bank said in a statement that it would not com